In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat

In [3]:
PATH = os.path.join("..", "content", "mnist-original.mat")

In [4]:
CLUSTERS = 856
EPOCHS = 10
IMG_SIZE = 28
SAMPLES = 60000

In [5]:
mnist = loadmat(PATH)
mnist_data, mnist_label = np.array(mnist["data"].T), np.array(mnist["label"][0].T)
train_mnist_data, test_mnist_data = mnist_data[:SAMPLES], mnist_data[SAMPLES:]
train_mnist_label, test_mnist_label = mnist_label[:SAMPLES], mnist_label[SAMPLES:]

MAKING THE TARGET MODEL

In [6]:
from sklearn.cluster import KMeans
from scipy.stats import mode

In [7]:
k_means = KMeans(n_clusters=CLUSTERS, n_init="auto", max_iter=EPOCHS, random_state=42)
k_means.fit(train_mnist_data)
pred_train_mnist_label = k_means.predict(train_mnist_data)

In [8]:
cluster_labels = np.zeros(CLUSTERS, dtype=int)
for i in range(CLUSTERS):
    mask = pred_train_mnist_label == i
    cluster_labels[i] = mode(train_mnist_label[mask], keepdims=True).mode[0]

In [10]:
pred_test_mnist_label = k_means.predict(test_mnist_data)
pred_test_mnist_label = cluster_labels[pred_test_mnist_label]

accuracy = np.mean(pred_test_mnist_label == test_mnist_label)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 94.01%


MAKING THE SURROGATE MODEL

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu
